# Volume Tracing

We will start with a rather simple example: A point light source and a spherical
detector, both submerged in a homogenous medium. This will introduce reoccurring
topics used in most simulations.

In [ ]:
# Uncomment this, if you run this on Google Colab
# !sudo apt-get install -y libnvidia-gl-550 vulkan-tools
# !pip install git+https://github.com/tkerscher/theia

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import hephaistos as hp
import hephaistos.pipeline as pl

import theia
import theia.units as u

# print device we run this on
print(hp.getCurrentDevice())

## Medium

In order to be able to trace light, we need to define the medium it propagates in.
Theia already gives us some building blocks to give as a head start.

We will model a dispersion free medium, that is its optical properties do not
depend on the wavelength, with a scattering distribution described by the
Henyey-Greenstein phase function.

In [ ]:
from theia.material import DispersionFreeMedium, HenyeyGreensteinPhaseFunction, MediumModel

class MediumModel(
    DispersionFreeMedium,
    HenyeyGreensteinPhaseFunction,
    MediumModel,
):
    def __init__(self, *, n, mu_a, mu_s, g) -> None:
        DispersionFreeMedium.__init__(self, n=n, ng=n, mu_a=mu_a, mu_s=mu_s)
        HenyeyGreensteinPhaseFunction.__init__(self, g)

    # name of the model used for referencing it later
    ModelName = "homogenous"

model = MediumModel(n=1.33, mu_a=0.05 / u.m, mu_s = 0.02 / u.m, g=0.2)
medium = model.createMedium()

Currently the medium is still on CPU. We need to make it available on GPU using `MaterialStore`.
In more advanced examples it will also handle materials, hence the name.

In [ ]:
from theia.material import MaterialStore

matStore = MaterialStore([], media=[medium])

## Light

Next, we define the light source. In `theia` light sources a split into a
`LightSource` that samples the direction and amount of emitted light, and a
`WavelengthSource` that samples the wavelength. This modularization was chosen
to allow more reusability.

Since in dispersion free media the wavelength does not matter, we use a constant
one. For the light source, as mentioned earlier, we will use a point light source.

In [ ]:
from theia.light import ConstWavelengthSource, SphericalLightSource

wavelength = ConstWavelengthSource(500.0 * u.nm)
light = SphericalLightSource(
    position=(1.0, 0.0, 0.0) * u.m, # position of the light source
    timeRange=(0.0, 10.0) * u.ns,   # time range light will be produced
    budget=1.0e5                    # How much light produced
)

You might have noticed, that the light budget does not have a unit. This is by
design, as the tracing simulation does not dictate one. Depending on the
context of your simulation the budget can for instance be photons or energy.

## Target

With our light settled, we now define our target. The tracer will test it for
intersections with the light ray during the tracing.

In this simulation this will be a simple spherical target.

In [ ]:
from theia.target import SphereTarget

target = SphereTarget(
    position=(-1.0, 0.0, 0.0) * u.m,
    radius=50.0 * u.cm,
)

## Response

The tracer itself does not know what to do with the hits it created.
Instead it passes them on to the response module.

Here we use a somewhat special response: `HistogramHitResponse`
First it asks a second `ValueResponse` to transform the hit to a single number
and timestamp. The response uses these to create a histogram on the GPU only
copying back the final histogram. In our case we use a simple uniform value
response that simply returns the received energy. (The cosine of the surface
normal was already taken care of by the tracer)

This has two advantages: We cannot only salvage the computational power of the
GPU, we also reduce the amount of data transferred between CPU and GPU. Since
programs running on the latter are typically bandwidth limited this actually
increases performance.

In [ ]:
from theia.response import HistogramHitResponse, UniformValueResponse

n_bins = 100
bin_size = 5.0 * u.ns

value = UniformValueResponse()
response = HistogramHitResponse(
    value,
    nBins=n_bins,
    binSize=bin_size,
)

## Random Number Generator

Last building block before we can create the tracer is the source of random
numbers.

In [ ]:
from theia.random import PhiloxRNG

rng = PhiloxRNG(key=42)

If no key or seed is given, a random one from the OS will be used.
Usually this is not desired, as it makes the result of your simulation
non-reproducible.

## Tracer

Finally, with all necessary building blocks created, we can define the tracer.

In [ ]:
from theia.trace import VolumeForwardTracer

batch_size = 512 * 1024 # how many light paths to sample per run
sample_coef = 0.05 / u.m # "scatter" length used for sampling path segment lengths
n_scatter = 10 # Number of scatter events to simulate for each path

tracer = VolumeForwardTracer(
    batch_size,
    light,
    target,
    wavelength,
    response,
    rng,
    scatterCoefficient=sample_coef,
    nScattering=n_scatter,
    # Do not forget this line! Otherwise we will trace in vacuum!
    medium=matStore.media["homogenous"],
)
# notify the rng to advance automatically between runs
rng.autoAdvance = tracer.nRNGSamples

## Pipeline + Scheduler

The building blocks need to be assembled into a pipeline, that handles the
submission of work to the GPU.

In [ ]:
pipeline = pl.Pipeline(tracer.collectStages())

The orchestration of processing the results of each batch on the CPU and running
batches on the GPU are handled by a `PipelineScheduler`. We can pass a process
function to it, that gets automatically called for each finished batch. The
scheduler will then wait for it to finish before issuing the next batch. However,
since the pipeline is double buffered, there is on batch in flight while the
previous one is processed allowing for some parallelism between GPU and CPU.

In [ ]:
hists = []

def process(config: int, task: int) -> None:
    # store a copy of the histogram in a list
    hists.append(response.result(config).copy())

scheduler = pl.PipelineScheduler(pipeline, processFn=process)

## Running the pipeline

Now everything is set up and we are ready to issue batches of work.

A single work submission is defined by a dictionary describing the state changes
of the stages in the pipeline. For now we only simulate a single setting, so
these dictionaries are empty

In [ ]:
n_runs = 20

# define 20 runs
tasks = [{}, ] * n_runs

# submit work to scheduler
scheduler.schedule(tasks)

After submitting the work, we are free to do something else on the CPU side.
Since we do not have anything to do right now, we will simply wait for the work
to finish.

In [ ]:
scheduler.wait()

## Finalizing the results

This was one of the simplest simulations possible. The only thing left to do is
to create some nice plots.

In [ ]:
# combine the individual histograms
final_hist = np.mean(hists, 0)

# create the time coordinates of the bins
t = np.arange(n_bins) * bin_size

In [ ]:
plt.figure()
plt.step(t, final_hist)
plt.yscale("log")
plt.show()